### Machine Learning for Physical Sciences
*pip install mlphys*

Author: Sanjaya Lohani

*Please report bugs at slohani@mlphys.com

Papers:

1.   Lohani, S., Lukens, J.M., Jones, D.E., Searles, T.A., Glasser, R.T. and Kirby, B.T., 2021. Improving application performance with biased distributions of quantum states. *Physical Review Research*, 3(4), p.043145. 

2.  Lohani, S., Searles, T. A., Kirby, B. T., & Glasser, R. T. (2021). On the Experimental Feasibility of Quantum State Reconstruction via Machine Learning. *IEEE Transactions on Quantum Engineering*, 2, 1–10. 

Collaborator: Joseph M. Lukens, Daniel E. Jones, Ryan T. Glasser, Thomas A. Searles, and Brian T. Kirby




In [ ]:
!pip install mlphys

In [1]:
import mlphys.deepqis.simulator.distributions as dist
import mlphys.deepqis.simulator.measurements as meas
import mlphys.deepqis.utils.Alpha_Measure as find_alpha
import mlphys.deepqis.utils.Concurrence_Measure as find_con
import mlphys.deepqis.utils.Purity_Measure as find_pm
import mlphys.deepqis.network.inference as inference
import mlphys.deepqis.utils.Fidelity_Measure as fm
import matplotlib.pyplot as plt

In [2]:
# Generating 100 test sets 
# Random measurements = 1000. 
#""" NOTE: The Random measurements illustrate the use of a single basis chosen 
# randomly at a time, which is different from the term "shots" used in NISQ. 
#"""

# 1. Bures and its tomography 
bures = dist.Bures(qs=2).sample_dm(100)
tomo_bures,_ = meas.Random_Measurements(qs=2, n_meas=1000).tomography_data(bures)

# 2. HS
hs = dist.Hilbert_Schmidt(qs=2).sample_dm(100)
tomo_hs,_ = meas.Random_Measurements(qs=2, n_meas=1000).tomography_data(hs)

#3. Haar random pure states
haar = dist.Haar_State(qs=2).sample_dm(100)
tomo_haar,_ = meas.Random_Measurements(qs=2, n_meas=1000).tomography_data(haar)



| To accelerate the simulation, General Scheme Projector file is created in a new projectors_general folder.
|Found projectors_general folder! Appending some files ...
| To accelerate the simulation, General Scheme Projector file is created in a new projectors_general folder.
|Found projectors_general folder! Appending some files ...
| To accelerate the simulation, General Scheme Projector file is created in a new projectors_general folder.
|Found projectors_general folder! Appending some files ...


### To demonstrate the efficacy of biased distributions, we have inculded only two alpha 

---

values at the moment. $\alpha = 0.1, 0.4$

##### 1. Reconstructing States using a pre-trained model with MA at $\alpha = 0.1$



In [4]:
pred_bures, _ = inference.fit(tomo_bures, alpha=0.1)
pred_hs, _ = inference.fit(tomo_hs, alpha=0.1)
pred_haar, model = inference.fit(tomo_haar, alpha=0.1)

print ('*'*100)

_, fid_bures_av = fm.Fidelity_Metric(bures, pred_bures)
print("Mean Fidelity for Bures: ", fid_bures_av)

_, fid_hs_av = fm.Fidelity_Metric(hs, pred_hs)
print("Mean Fidelity for HS: ", fid_hs_av)

_, fid_haar_av = fm.Fidelity_Metric(haar, pred_haar)
print("Mean Fidelity for Haar: ", fid_haar_av)
print ('*'*100)


****************************************************************************************************
Mean Fidelity for Bures:  tf.Tensor(0.908354743815255, shape=(), dtype=float64)
Mean Fidelity for HS:  tf.Tensor(0.8860056817796851, shape=(), dtype=float64)
Mean Fidelity for Haar:  tf.Tensor(0.9741458761737515, shape=(), dtype=float64)
****************************************************************************************************


##### 2. Reconstructing States using a pre-trained model with MA at $\alpha = 0.4$

In [6]:
pred_bures, _ = inference.fit(tomo_bures, alpha=0.4)
pred_hs, _ = inference.fit(tomo_hs, alpha=0.4)
pred_haar, model = inference.fit(tomo_haar, alpha=0.4)

print ('*'*100)

_, fid_bures_av = fm.Fidelity_Metric(bures, pred_bures)
print("Mean Fidelity for Bures: ", fid_bures_av)

_, fid_hs_av = fm.Fidelity_Metric(hs, pred_hs)
print("Mean Fidelity for HS: ", fid_hs_av)

_, fid_haar_av = fm.Fidelity_Metric(haar, pred_haar)
print("Mean Fidelity for Haar: ", fid_haar_av)
print ('*'*100)


****************************************************************************************************
Mean Fidelity for Bures:  tf.Tensor(0.9454871408272887, shape=(), dtype=float64)
Mean Fidelity for HS:  tf.Tensor(0.9442138892237649, shape=(), dtype=float64)
Mean Fidelity for Haar:  tf.Tensor(0.9397807952668203, shape=(), dtype=float64)
****************************************************************************************************


### View the model used. 


In [7]:
model.summary()

Model: "ARL_Nets"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Tomography_Measurements (Inp [(None, 6, 6, 1)]         0         
_________________________________________________________________
First_CONV (Conv2D)          (None, 6, 6, 64)          320       
_________________________________________________________________
First_MAXPOOL (MaxPooling2D) (None, 3, 3, 64)          0         
_________________________________________________________________
Second_CONV (Conv2D)         (None, 3, 3, 64)          16448     
_________________________________________________________________
FLATTEN (Flatten)            (None, 576)               0         
_________________________________________________________________
First_DENSE (Dense)          (None, 3000)              1731000   
_________________________________________________________________
First_DROPOUT (Dropout)      (None, 3000)              0  

### The tensorflow model can also be directly imported from the 'load' module. After that it can be used in fine tuning any other network-settings and training scenarios.

In [8]:
import tensorflow as tf
from mlphys.deepqis.utils import Extract_Net
model_h5 = inference.load(alpha=0.4)
model = tf.keras.models.load_model(model_h5, custom_objects={'ErrorNode':Extract_Net.ErrorNode, 
                                        'PredictDensityMatrix':Extract_Net.PredictDensityMatrix})
model.summary()

Model: "ARL_Nets"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Tomography_Measurements (Inp [(None, 6, 6, 1)]         0         
_________________________________________________________________
First_CONV (Conv2D)          (None, 6, 6, 64)          320       
_________________________________________________________________
First_MAXPOOL (MaxPooling2D) (None, 3, 3, 64)          0         
_________________________________________________________________
Second_CONV (Conv2D)         (None, 3, 3, 64)          16448     
_________________________________________________________________
FLATTEN (Flatten)            (None, 576)               0         
_________________________________________________________________
First_DENSE (Dense)          (None, 3000)              1731000   
_________________________________________________________________
First_DROPOUT (Dropout)      (None, 3000)              0  